In [3]:
%pip install --upgrade torch transformers peft

  Using cached setuptools-74.0.0-py3-none-any.whl.metadata (6.7 kB)
   ---------------------------------------- 0.0/197.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/197.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/197.8 MB 3.8 MB/s eta 0:00:53
   ---------------------------------------- 0.4/197.8 MB 4.2 MB/s eta 0:00:48
   ---------------------------------------- 0.6/197.8 MB 4.6 MB/s eta 0:00:43
   ---------------------------------------- 0.7/197.8 MB 5.1 MB/s eta 0:00:39
   ---------------------------------------- 0.8/197.8 MB 4.6 MB/s eta 0:00:43
   ---------------------------------------- 0.8/197.8 MB 4.6 MB/s eta 0:00:43
   ---------------------------------------- 1.0/197.8 MB 3.9 MB/s eta 0:00:52
   ---------------------------------------- 1.1/197.8 MB 3.7 MB/s eta 0:00:54
   ---------------------------------------- 1.1/197.8 MB 3.7 MB/s eta 0:00:54
   ---------------------------------------- 1.1/197.8 MB 3.7 MB/s eta 0:00:54
   -------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1+cu121, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1+cu121, but you have torch 2.4.0 which is incompatible.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset

# Load model and tokenizer
model_name = "croissantllm/CroissantLLMBase"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Try
inputs = tokenizer("Romain Lhotte is ", return_tensors="pt").to(model.device)
tokens = model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60, temperature=0.3)
print(tokenizer.decode(tokens[0]))

# Prepare the text for LoRA fine-tuning
text = "Romain Lhotte is a Software Engineer at APHP, France"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

# LoRA Configuration
lora_config = LoraConfig(
    r=1,  # Rank of the decomposition
    lora_alpha=16,  # Scaling factor for the LoRA weights
    lora_dropout=0.1,  # Dropout rate applied to LoRA weights
    target_modules=["q_proj", "v_proj"],  # Which layers to apply LoRA to
)

# Wrap the original model with LoRA
model = get_peft_model(model, lora_config)

# Prepare the input data as a custom dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids
        self.labels = input_ids.clone()  # Labels are the same as input_ids for causal LM training

    def __len__(self):
        return 1  # Only one example in this case

    def __getitem__(self, idx):
        return {"input_ids": self.input_ids.squeeze(), "labels": self.labels.squeeze()}

# Create the dataset
train_dataset = CustomDataset(inputs['input_ids'])

# Prepare the training arguments
training_args = TrainingArguments(
    output_dir="./lora_croissantllm",
    per_device_train_batch_size=1,
    num_train_epochs=10,  # Adjust based on your dataset size
    logging_dir="./logs",
    logging_steps=1,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Try again
inputs = tokenizer("Romain Lhotte is ", return_tensors="pt").to(model.device)
tokens = model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60, temperature=0.3)
print(tokenizer.decode(tokens[0]))

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Romain Lhotte is 25 years old and lives in the city of Toulouse. He is a student in the Master of Science in Management of Innovation and Entrepreneurship at the University of Toulouse 1 Capitole.	Romain Lhotte a 25 ans et habite la ville de Toulouse. Il est étudiant en Master of Science in Management of Innovation and Entrepreneurship à l’Université de Toulouse 1 Capitole.</s>


 10%|█         | 1/10 [01:25<12:46, 85.15s/it]

{'loss': 4.2762, 'grad_norm': 9.295404434204102, 'learning_rate': 4.5e-05, 'epoch': 1.0}


 20%|██        | 2/10 [02:49<11:17, 84.74s/it]

...

{'loss': 3.9241, 'grad_norm': 10.534070014953613, 'learning_rate': 5e-06, 'epoch': 9.0}


100%|██████████| 10/10 [14:01<00:00, 83.46s/it]

{'loss': 3.8698, 'grad_norm': 11.138350486755371, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 10/10 [14:01<00:00, 84.17s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'train_runtime': 841.6751, 'train_samples_per_second': 0.012, 'train_steps_per_second': 0.012, 'train_loss': 4.039868545532227, 'epoch': 10.0}
<s> Romain Lhotte is 28 years old and is a French photographer. He was born in Paris and studied at the Ecole Nationale Supérieure de la Photographie in Arles. He has been working for the past 3 years as a freelance photographer in Paris and in the South of France. His work has been published in numerous magazines and newspapers such as Le Monde, Libération, Le Figaro, Le Point, Le Nouvel Observateur, L'Express
